# 07 - FX Option Pricing (Garman-Kohlhagen)

Price FX options, verify parity, and optionally invert implied volatility.

In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import pandas as pd

from src.config import config_dict
from src.fx_options import gk_call_price, gk_put_price, gk_put_call_parity_residual, gk_implied_vol

cfg = config_dict(fast_mode=True)


In [ ]:
strikes = [1.00, 1.05, 1.10, 1.15, 1.20]
rows = []
for K in strikes:
    call = gk_call_price(cfg['S0_FX'], K, cfg['RD'], cfg['RF'], cfg['SIGMA_FX'], cfg['T_FX'])
    put = gk_put_price(cfg['S0_FX'], K, cfg['RD'], cfg['RF'], cfg['SIGMA_FX'], cfg['T_FX'])
    residual = gk_put_call_parity_residual(cfg['S0_FX'], K, cfg['RD'], cfg['RF'], cfg['SIGMA_FX'], cfg['T_FX'])
    iv = gk_implied_vol(call, cfg['S0_FX'], K, cfg['RD'], cfg['RF'], cfg['T_FX'], 'call')
    rows.append({'strike': K, 'call': call, 'put': put, 'parity_residual': residual, 'implied_vol_call': iv})

fx_df = pd.DataFrame(rows)
fx_df.to_csv(ROOT / 'results' / 'tables' / 'fx_parity_checks.csv', index=False)
fx_df
